In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    MessagesPlaceholder,
)
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough

"""
앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.
"""

llm = ChatOpenAI(
    # model_name="gpt-5-nano",
    model_name="gpt-3.5-turbo",
    temperature=0.1,
)

# 메모리
memory = ConversationBufferMemory(
    llm=llm, memory_key="history", return_messages=True
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


# 영화-이모티콘 예시 리스트
examples = [
    {"movie": "탑건", "emojis": "🛩️👨‍✈️🔥"},
    {"movie": "대부", "emojis": "👨‍👨‍👦🔫🍝"},
]

# example_prompt 문자열 템플릿 정의
example_prompt_str = """
Human: {movie}
AI: {emojis}
"""
example_prompt = PromptTemplate.from_template(example_prompt_str)

few_shot_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: {movie} \nAI:",
    input_variables=["movie"],
    prefix=(
        "영화 제목을 보고 해당 영화를 나타내는 정확히 3개의 이모티콘으로만"
        " 응답하세요. 다른 텍스트나 설명 없이 이모티콘만 답변하세요.\n\n예시:"
    ),
)

chain = RunnablePassthrough.assign(history=load_memory) | few_shot_prompt | llm


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(f'"{movie}" -> "{result.content}"')


# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI that remembers movie questions."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)
# 체인 구성
chain2 = RunnablePassthrough.assign(history=load_memory) | prompt | llm

invoke_chain("벤허")
invoke_chain("로마의휴일")

"벤허" -> "👻🏚️👩‍👧"
"로마의휴일" -> "👩‍🦰🏛️🛵"


In [29]:
chain2.invoke({"question": "내가 이전에 물어본 영화들이 뭐였지?"})

AIMessage(content='당신이 이전에 물어본 영화는 "벤허"와 "로마의 휴일"입니다.')